In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [22]:
plt.rcParams.update({"text.usetex": False,
                     "font.size": 20,
                     "pdf.fonttype": 42,
                     "ps.fonttype": 42})

In [23]:
target_latency = 140

In [24]:
# local_df = pd.read_csv("../measurements/local_long/map.csv")
# root_folder = "../measurements/direct_split_quantize/"
# mag_folder = "../measurements_vidvrd/test_ltl_15/"

local_df = pd.read_csv("../measurements_vidvrd/local_test/map.csv")
root_folder = "../measurements_vidvrd/direct_split_quantize/"
mag_folder = "../measurements_vidvrd/test_ltl_15/"

# local_df = pd.read_csv("../measurements_bev/local_test/map.csv")
# root_folder = "../measurements_bev/direct_split_quantize/"
# mag_folder = "../measurements_bev/test_ltl_15/"


map_df = pd.read_csv(root_folder+"map.csv")
cha_df = pd.read_csv(mag_folder+"characteristic.csv")
mag_df = pd.read_csv(mag_folder+"manager.csv")
tim_df = pd.read_csv(root_folder+"time.csv")

In [25]:
tim_df

,frame_id,head_time,serialize_time,tail_time
0,1,242.832031,68.167458,1944.066772
1,2,10.671840,3.011136,14.581824
2,3,10.552640,2.475840,12.389568
3,4,10.413312,2.173504,11.973856
4,5,10.459680,2.693024,16.027937
...,...,...,...,...
395,396,10.295936,1.906848,11.630432
396,397,10.354592,1.985184,11.752608
397,398,10.271104,1.817472,11.560096
398,399,10.232768,1.845024,11.602368


In [26]:
drops = []

for i in range(400):
    try:
        d = mag_df[mag_df["frame_id"]==i+1]["drop"].tolist()[0]
        if d >0.4:
            d=0.4
        drops.append(d)
    except:
        drops.append(drops[-1])


df = pd.DataFrame()
df["traget_drops"] = drops

In [27]:
tim_df["head_est"] = tim_df["head_time"]*10
tim_df["ser_est"] = tim_df["serialize_time"]*10
tim_df["uav"] = tim_df["head_est"]+tim_df["ser_est"] 

In [29]:
tim_df[200:].max()

frame_id          400.000000
head_time          10.807776
serialize_time      3.124672
tail_time          16.109247
head_est          108.077765
ser_est            31.246719
uav               134.466562
dtype: float64

In [40]:
# test_df = pd.read_csv("../measurements/real_bw_test_15fps/characteristic.csv")
test_df = pd.read_csv("../measurements_bev/static_test_50/characteristic.csv")

In [42]:
test_df[200:]["datasize_est"].max()

12711